In [1]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
movie_data = load_files(r"D:\Master Thesis\txt_sentoken")
movie_reviews, y = movie_data.data, movie_data.target
print(movie_reviews[4])
print(movie_data.target[4])

b"kolya is one of the richest films i've seen in some time . \nzdenek sverak plays a confirmed old bachelor ( who's likely to remain so ) , who finds his life as a czech cellist increasingly impacted by the five-year old boy that he's taking care of . \nthough it ends rather abruptly-- and i'm whining , 'cause i wanted to spend more time with these characters-- the acting , writing , and production values are as high as , if not higher than , comparable american dramas . \nthis father-and-son delight-- sverak also wrote the script , while his son , jan , directed-- won a golden globe for best foreign language film and , a couple days after i saw it , walked away an oscar . \nin czech and russian , with english subtitles . \n"
1


In [3]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(movie_reviews)):
    # Remove all the special characters
    #document = re.sub(r'\W', ' ', str(movie_reviews[sen]))
    
    # remove all single characters
    #document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    #document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    #document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    #document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    #document = document.lower()
    
    # Lemmatization
    #document = document.split()

    #document = [stemmer.lemmatize(word) for word in document]
    #document = ' '.join(document)
    
    #documents.append(document)
    documents.append(str(movie_reviews[sen]))

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents)

In [5]:
#Term frequency = (Number of Occurrences of a word)/(Total words in the document)
#IDF(word) = Log((Total number of documents)/(Number of documents containing the word))

from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [8]:
y_pred = classifier.predict(X_test)

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[131  77]
 [ 63 129]]
              precision    recall  f1-score   support

           0       0.68      0.63      0.65       208
           1       0.63      0.67      0.65       192

    accuracy                           0.65       400
   macro avg       0.65      0.65      0.65       400
weighted avg       0.65      0.65      0.65       400

0.65


In [10]:
with open('text_classifier', 'wb') as picklefile:
    pickle.dump(classifier,picklefile)

In [11]:
import eli5
eli5.show_weights(classifier, top=10)

C:\Users\Marc\Anaconda3\envs\eli5\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Marc\Anaconda3\envs\eli5\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weight,Feature
0.0923,x104
0.0424,x1482
0.0245,x142
0.0221,x1435
0.0211,x1105
0.0200,x100
0.0156,x982
0.0137,x1282
0.0134,x801
0.0133,x1485


In [12]:
eli5.show_weights(classifier, vec=vectorizer, top=10)

Weight,Feature
0.0923,bad
0.0424,worst
0.0245,boring
0.0221,wasted
0.0211,ridiculous
0.0200,awful
0.0156,performances
0.0137,summer
0.0134,minutes
0.0133,would


In [13]:
import pixiedust

In [14]:
doc = documents[105]
#, show=['feature_importances','decision_tree','method','description']
import eli5
eli5.explain_prediction(classifier, doc, vec=vectorizer,target_names=['pos','neg'])

Explanation(estimator="DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=None,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=None, splitter='best')", description='\nFeatures with largest coefficients.\n\nFeature weights are calculated by following decision paths in trees\nof an ensemble (or a single tree for DecisionTreeClassifier).\nEach node of the tree has an output score, and contribution of a feature\non the decision path is how much the score changes from parent to child.\nWeights of all features sum to the output score or proba of the estimator.\n\nCaveats:\n1. Feature weights just show if the feature contributed positively or\n   negatively to the final score, and does not show how increasing or\n   decreasing the feature value will change the prediction.\n2. In some cases, feature weight can be close to zero for an important feature.\n   For example, in a single tree that computes XOR function, the feature at the\n   top of the tree will have zero weight because expected scores for both\n   branches are equal, so decision at the top feature does not change the\n   expected score. For an ensemble predicting XOR functions it might not be\n   a problem, but it is not reliable if most trees happen to choose the same\n   feature at the top.\n', error=None, method='decision path', is_regression=False, targets=[TargetExplanation(target='neg', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='quite', weight=0.7719298245614035, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=0.505, std=None, value=1.0), FeatureWeight(feature='bad', weight=0.08465804837364466, std=None, value=0.0), FeatureWeight(feature='worst', weight=0.0283790869048699, std=None, value=0.0), FeatureWeight(feature='boring', weight=0.02549898639448922, std=None, value=0.0), FeatureWeight(feature='ridiculous', weight=0.022175536881419267, std=None, value=0.0), FeatureWeight(feature='wasted', weight=0.019162291025408895, std=None, value=0.0), FeatureWeight(feature='awful', weight=0.018182818979119997, std=None, value=0.0)], neg=[FeatureWeight(feature='minutes', weight=-0.2552842497670084, std=None, value=1.0), FeatureWeight(feature='perfectly', weight=-0.06459505749938832, std=None, value=0.0), FeatureWeight(feature='everything', weight=-0.0621218233158532, std=None, value=0.0), FeatureWeight(feature='latest', weight=-0.05061834915156743, std=None, value=0.0), FeatureWeight(feature='performances', weight=-0.04236711338653809, std=None, value=0.0)], pos_remaining=0, neg_remaining=0), proba=1.0, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [16]:
eli5.show_prediction(classifier, doc, vec=vectorizer,
                     target_names=['pos','neg'],
                     targets=['neg'])